Import Statements

In [282]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import numpy as np

KaggleApi Settings

In [283]:
api = KaggleApi()
api.authenticate()

Import Datasets with Kaggle.API

In [284]:
api.dataset_download_file('thedevastator/global-fossil-co2-emissions-by-country-2002-2022',
                          file_name='GCB2022v27_MtCO2_flat.csv')
with zipfile.ZipFile('GCB2022v27_MtCO2_flat.csv.zip', 'r') as zipref:
    zipref.extractall()

In [285]:
api.dataset_download_file('russellyates88/suicide-rates-overview-1985-to-2016',
                          file_name='master.csv')
with zipfile.ZipFile('master.csv.zip', 'r') as zipref:
    zipref.extractall()

In [286]:
api.dataset_download_file('yapwh1208/countries-gdp-2012-to-2021',
                          file_name='GDP.csv')

False

In [287]:
api.dataset_download_file('folaraz/world-countries-and-continents-details',
                          file_name='Countries Longitude and Latitude.csv')

False

Read and Filter DataFiles

In [288]:
df_emissions = pd.read_csv("GCB2022v27_MtCO2_flat.csv", sep=",")

new_df_emissions = df_emissions[(df_emissions['Year'] >= 2006) & (df_emissions['Year'] <= 2016)]
new_df_emissions.rename(columns={'Year':'year', 'Country':'country'}, inplace=True)
new_df_emissions = new_df_emissions.drop(['Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita'], axis=1)

/var/folders/xv/m8qj2dh50mqf5mb38h6d9kyh0000gn/T/ipykernel_47986/2350370852.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_emissions.rename(columns={'Year':'year', 'Country':'country'}, inplace=True)


In [289]:
df_suicide = pd.read_csv("master.csv", sep=",")

new_df_suicide = df_suicide[(df_suicide['year'] >= 2006) & (df_suicide['year'] <= 2016)]
new_df_suicide = new_df_suicide.groupby(['country', 'year']).sum(). reset_index()
new_df_suicide = new_df_suicide.drop(['HDI for year', 'gdp_per_capita ($)'], axis=1)

/var/folders/xv/m8qj2dh50mqf5mb38h6d9kyh0000gn/T/ipykernel_47986/2259007338.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  new_df_suicide = new_df_suicide.groupby(['country', 'year']).sum(). reset_index()


In [290]:
df_gdp = pd.read_csv("GDP.csv", sep=",")

new_df_gdp = df_gdp[['Country Name', 'Country Code', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']]
new_df_gdp = new_df_gdp.melt(id_vars=["Country Name", "Country Code"], var_name="year")
new_df_gdp['year'] = new_df_gdp['year'].astype('int64')
new_df_gdp.rename(columns={'Country Name':'country', 'value':'GDP_Value'}, inplace=True)

In [291]:
df_allCountries = pd.read_csv("Countries%20Longitude%20and%20Latitude.csv", sep=",")

In [292]:
df_countries = pd.read_excel("countries.xlsx", sheet_name = "Tabelle1")

kombinieren


In [293]:
merged_df_emissions_suicide = pd.merge(new_df_emissions, new_df_suicide, on=['country', 'year'])

In [298]:
merged_df_emissions_gdp = pd.merge(new_df_emissions, new_df_gdp, on=['country', 'year'])
merged_df_emissions_gdp = merged_df_emissions_gdp.drop(['Country Code'], axis=1)

In [295]:
merged_df_suicide_gdp = pd.merge(new_df_suicide, new_df_gdp, on=['country', 'year'])

Use of Python built-in data structures (lists, dictionaries, sets, tuples)

Conditional statements, loop control statements and loops

In [296]:
merged_df_emissions_suicide = merged_df_emissions_suicide.fillna(0)
merged_df_emissions_gdp = merged_df_emissions_gdp.fillna(0)
merged_df_suicide_gdp = merged_df_suicide_gdp.fillna(0)

Use of tables, vizualizations/graphics for data exploration

Integration of a statistical analyses (correlation analysis, statical tests)